In [1]:
import requests
import pandas as pd
import re
from fuzzywuzzy import fuzz
import string
from tqdm import tqdm
from IPython.display import clear_output

from decouple import config

Phase 1
1. Pass extracted list of names into autocomplete function
2. From API return Crunchbase name, country, link, permalink, UUID
3. Fuzzy match
    a. Do ratio match. If >90, directly return, else go to b
    b. Do token_set_ratio
    b. If len(word)>1, remove last word and token_set_ratio again. Return if >90
4. Manual review

Phase 2
- Using matched companies, pass UUID/permalink into entity lookup function
- Return highest funding round, founders identifiers

In [2]:
# Environment Variables
CB_API_KEY = config("CB_API_KEY")
url = f"https://api.crunchbase.com/api/v4/searches/organizations"

In [176]:
raw = pd.read_csv("esd.csv")

In [177]:
base = "https://api.crunchbase.com/api/v4"

In [179]:
# Get a dictionary of Alpha-2 country code to country name since Crunchbase uses Alpha-2 code
# e.g. {"DE": "Germany", "SG": "Singapore"}

country_map = pd.read_html("https://www.iban.com/country-codes")[0]
country_map = country_map.iloc[:,:2]
country_map.columns = ['country','code']
country_map = country_map.set_index("code").to_dict()['country']

In [180]:
def namecheck(rowdata):
    """
    This function performs a fuzzy match between the top results using the Crunchbase autocomplete function and the target startup name
    (i.e. how similar the names are)
    
    If there is a match, return a tuple in the format (startup_name, startup_uuid, startup_description)
    
    If there are no similar matches, return a list of top results found in the same country as the target startup in the format
    [(startup_1_name, startup_1_uuid, startup_1_description), (startup_2_name,...)]
    

    """
    name = rowdata["name"]
    action = "/autocompletes"
    url = base+action
    results = requests.get(url,headers={"X-cb-user-key":CB_API_KEY},params={"collection_ids":"organizations","query":f"{name}","limit":10}).json()
    candidates = [(e['identifier']['value'],e['identifier']['uuid'],e['short_description'])for e in results['entities']]
    for candidate in candidates:
        if fuzz.token_set_ratio(candidate[0],name)>90:
            return candidate
        else:
            # Remove punctuation
            name = name.translate(str.maketrans(" "," ",string.punctuation))
            splitname = name.split()
            if len(splitname)>1:
                splitname = " ".join(splitname[:-1])
                if fuzz.token_set_ratio(candidate[0],splitname)>95:
                    return candidate
    
    candidates_2 = []
    
    for candidate in candidates:
        uuid = candidate[1]
        action = f"/entities/organizations/{uuid}"
        url = base+action
        results = requests.get(url,headers={"X-cb-user-key":CB_API_KEY},params={"card_ids":["headquarters_address"]}).json()['cards']['headquarters_address']
        if len(results):
            country = results[0]['country_code']
            if country_map[country]==rowdata["country"]:
                candidates_2.append(candidate)
        else:
            candidates_2.append(candidate)
    if len(candidates_2):
        return candidates_2

In [182]:
tqdm.pandas()

raw['results'] = raw.progress_apply(namecheck,axis=1)

100%|██████████| 615/615 [39:50<00:00,  3.89s/it]  


In [184]:
raw['results'].apply(type).value_counts()

<class 'tuple'>       370
<class 'NoneType'>    135
<class 'list'>        110
Name: results, dtype: int64

In [186]:
df = raw.copy()
df['results'] = df['results'].apply(lambda x: None if isinstance(x,list) else x)

In [185]:
# This cell allows the user to perform a manual check of the unclear startups
# Running this cell will print the target startup's details along with the details of possible startups
# The user inputs either the index number of the correct startup or blank string to skip (if there are no correct startups)

df = raw.copy()

unclear = df[df['results'].apply(lambda x: isinstance(x,list))]

counter = 0
for r,row in tqdm(unclear.iterrows()):
    counter += 1
    print(f"{counter}/{len(unclear)} records")
    print(row['name'])
    print(row['summary'])
    print()
    for n,candidate in enumerate(row['results'][:10]):
        print(f"{n}: "+candidate[0]+":\t"+candidate[2])
    clear_output(wait=True)
    correct = input("Which one is it?")
    if correct=="":
        df.at[r,'results'] = None
    else:
        df.at[r,'results'] = row['results'][int(correct)]

22it [03:00,  8.21s/it]


KeyboardInterrupt: Interrupted by user

In [402]:
def phase2(rowdata):
    """
    
    """
    uuid = rowdata["results"][1]
    action = f"/entities/organizations/{uuid}"
    url = base+action
    results = requests.get(url,headers={"X-cb-user-key":api_key},params={"card_ids":"fields,founders,jobs"}).json()["cards"]
    
    # Check if funding eligible
    result_dict = {"website":None,"funding":None, "founders":None}
    if "website_url" in results["fields"]:
        result_dict["website"] = results["fields"]["website_url"]
    
    if "last_funding_type" in results["fields"]:
        if not bool(re.match(r"(series_[b-z])|(.*ipo.*)",results["fields"]["last_funding_type"])):
            result_dict["funding"] = results["fields"]["last_funding_type"]
        else:
            return None
    else:
        result_dict["funding"] = "UNKNOWN"
    
    # Get list of employee uuids, to get title and starting_date
    employees = {}
    for p in results['jobs']:
        title = None if 'title' not in p else p['title']
        started = None if 'started_on' not in p else p['started_on']['value'][:4]
        employees[p["person_identifier"]["uuid"]] = (title,started)
    
    # Get details of founders
    if "founders" in results and results["founders"]:
        founders = []
        for p in results["founders"]:
            founder_name = p["identifier"]["value"]
            founder_uuid = p["identifier"]["uuid"]
            linkedin = None if "linkedin" not in p else p["linkedin"]["value"]
            title,started = None,None
            if founder_uuid in employees:
                title,started = employees[founder_uuid]
            founders.append((founder_name,founder_uuid,linkedin,title,started))
    
    # if "jobs" in results and results["jobs"]:
    #     founders = []
    #     for p in results["jobs"]:
    #         if 'founder' in p['title'].lower() and p['is_current']:
    #             name = p["person_identifier"]["value"]
    #             person_uuid = p["person_identifier"]["uuid"]
    #             title = None if "title" not in p else p["title"]
    #             founders.append((p['person_identifier']['value'],p['person_identifier']['uuid'],
    #                             p['title'],p['started_on']['value']))
        result_dict["founders"] = founders
    return result_dict
    

In [403]:
found = df.dropna(subset=["results"])
found['results2'] = found.progress_apply(phase2,axis=1)

100%|██████████| 370/370 [06:29<00:00,  1.05s/it]
/var/folders/p_/scs49c6d5bz5kvl2jkkbnqzw0000gn/T/ipykernel_30395/759843292.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  found['results2'] = found.progress_apply(phase2,axis=1)


In [407]:
found['website'] = found['results2'].apply(lambda x: x['website'] if x else x)
found['funding'] = found['results2'].apply(lambda x: x['funding'] if x else x)
found['founders'] = found['results2'].apply(lambda x: x['founders'] if x else x)
found["founders"] = found["founders"].apply(lambda x: "\r".join([", ".join([e for e in p if e]) for p in x]) if x else x)
found = found.drop('results2',axis=1)

/var/folders/p_/scs49c6d5bz5kvl2jkkbnqzw0000gn/T/ipykernel_30395/3142160271.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  found['website'] = found['results2'].apply(lambda x: x['website'] if x else x)
/var/folders/p_/scs49c6d5bz5kvl2jkkbnqzw0000gn/T/ipykernel_30395/3142160271.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  found['funding'] = found['results2'].apply(lambda x: x['funding'] if x else x)
/var/folders/p_/scs49c6d5bz5kvl2jkkbnqzw0000gn/T/ipykernel_30395/3142160271.py:3: SettingWithCopyW

In [423]:
found.to_excel("esd_found.xlsx",index=False)
df[df['results'].isnull()].to_excel("esd_not_found.xlsx",index=False)

In [367]:
# Test cell
uuid = "f57b9762-20eb-4098-947f-dfa8c4a0db82"
action = f"/entities/organizations/{uuid}"
url = base+action
result = requests.get(url,headers={"X-cb-user-key":api_key},params={"card_ids":"fields,jobs,founders"}).json()